using ITensors
let
    N=2
    Sites = siteinds("S=1/2",N)
    
    os = OpSum()
    for j=1:N-1
        os += "Sz",j,"Sz",j+1
        os += 1/2,"S+",j,"S-",j+1
        os += 1/2,"S-",j,"S+",j+1
    end
    H = MPO(os,Sites)
    
    psi0 = randomMPS(Sites,1)
    
    nsweeps = 1
    maxdim = [10,20,100,100,200]
    cutoff = [1E-10]
    
    energy, psi = dmrg(H,psi0; nsweeps, maxdim, cutoff)
    
    return
end

using ITensors
let
  N = 200
  sites = siteinds("S=1/2",N)

  os = OpSum()
  for j=1:N-1
    os += "Sz",j,"Sz",j+1
    os += 1/2,"S+",j,"S-",j+1
    os += 1/2,"S-",j,"S+",j+1
  end
  H = MPO(os,sites)

  psi0 = randomMPS(sites,1)

  nsweeps = 1
  maxdim = [10,20,100,100,200]
  cutoff = [1E-10]

  energy, psi = dmrg(H,psi0; nsweeps, maxdim, cutoff)

  return
end

In [3]:
σᶻ = 0.5*[1 0;0 -1] #pauli matrices
σˣ = 0.5*[0 1; 1 0]

kron(σᶻ,σᶻ) #this is the matrix of the tensor product σᶻᵢ ⊗ σᶻⱼ


4×4 Matrix{Float64}:
 0.25   0.0    0.0    0.0
 0.0   -0.25   0.0   -0.0
 0.0    0.0   -0.25  -0.0
 0.0   -0.0   -0.0    0.25

In [4]:
⊗(x,y) = kron(x,y) 
σᶻ ⊗ σᶻ            #equivalent to kron()


4×4 Matrix{Float64}:
 0.25   0.0    0.0    0.0
 0.0   -0.25   0.0   -0.0
 0.0    0.0   -0.25  -0.0
 0.0   -0.0   -0.0    0.25

In [5]:
id = [1 0;0 1]

2×2 Matrix{Int64}:
 1  0
 0  1

In [6]:
g=1
J=1

H = - σᶻ⊗σᶻ⊗id⊗id - id⊗σᶻ⊗σᶻ⊗id - id⊗id⊗σᶻ⊗σᶻ - σᶻ⊗id⊗id⊗σᶻ
H -= g*(σˣ⊗id⊗id⊗id + id⊗σˣ⊗id⊗id + id⊗id⊗σˣ⊗id + id⊗id⊗id⊗σˣ)
H = J*H

16×16 Matrix{Float64}:
 -1.0  -0.5  -0.5  -0.0  -0.5  -0.0  …  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.5   0.0  -0.0  -0.5  -0.0  -0.5     -0.0   0.0  -0.0   0.0  -0.0   0.0
 -0.5  -0.0   0.0  -0.5  -0.0  -0.0     -0.5   0.0  -0.0  -0.0   0.0   0.0
 -0.0  -0.5  -0.5   0.0  -0.0   0.0      0.0  -0.5  -0.0   0.0   0.0   0.0
 -0.5  -0.0  -0.0  -0.0   0.0  -0.5     -0.0  -0.0  -0.5   0.0   0.0   0.0
 -0.0  -0.5  -0.0   0.0  -0.5   1.0  …  -0.0   0.0   0.0  -0.5   0.0   0.0
 -0.0  -0.0  -0.5   0.0  -0.5   0.0      0.0   0.0   0.0   0.0  -0.5   0.0
 -0.0   0.0   0.0  -0.5   0.0  -0.5      0.0   0.0   0.0   0.0   0.0  -0.5
 -0.5  -0.0  -0.0  -0.0  -0.0  -0.0     -0.5   0.0  -0.5   0.0   0.0   0.0
 -0.0  -0.5  -0.0   0.0  -0.0   0.0      0.0  -0.5   0.0  -0.5   0.0   0.0
 -0.0  -0.0  -0.5   0.0  -0.0  -0.0  …   1.0  -0.5   0.0   0.0  -0.5   0.0
 -0.0   0.0   0.0  -0.5  -0.0   0.0     -0.5   0.0   0.0   0.0   0.0  -0.5
 -0.0  -0.0  -0.0  -0.0  -0.5   0.0      0.0   0.0   0.0  -0.5  -0.5   0.0
 -

In [7]:
function TransverseFieldIsing(;N,J,g)
    id = [1 0;0 1]
    σˣ = [0 1;1 0]
    σᶻ = [1 0;0 -1]
    
    #Constructing a vector of operators: [σᶻ, σᶻ, id, id, ...]
    first_term_ops = fill(id, N)
    first_term_ops[1] = σᶻ
    first_term_ops[2] = σᶻ
    
    #Construction a vector of operators: [σˣ, id, ...]
    second_term_ops = fill(id, N)
    second_term_ops[1] = σˣ
    
    H = zeros(Int, 2^N, 2^N)
    for i in 1:N
        #tensor multiply all the operators
        H -= foldl(⊗, first_term_ops)
        #cylicly permute the operator array
        first_term_ops = circshift(first_term_ops,1)
    end
    
    for i in 1:N
        H -= g*foldl(⊗, second_term_ops)
        second_term_ops = circshift(second_term_ops,1)
    end
    H = J*H
    H
end

TransverseFieldIsing (generic function with 1 method)

In [8]:
TransverseFieldIsing(N=8, J=1, g=1)

256×256 Matrix{Int64}:
 -8  -1  -1   0  -1   0   0   0  -1  …   0   0   0   0   0   0   0   0   0
 -1  -4   0  -1   0  -1   0   0   0      0   0   0   0   0   0   0   0   0
 -1   0  -4  -1   0   0  -1   0   0      0   0   0   0   0   0   0   0   0
  0  -1  -1  -4   0   0   0  -1   0      0   0   0   0   0   0   0   0   0
 -1   0   0   0  -4  -1  -1   0   0      0   0   0   0   0   0   0   0   0
  0  -1   0   0  -1   0   0  -1   0  …   0   0   0   0   0   0   0   0   0
  0   0  -1   0  -1   0  -4  -1   0      0   0   0   0   0   0   0   0   0
  0   0   0  -1   0  -1  -1  -4   0      0   0   0   0   0   0   0   0   0
 -1   0   0   0   0   0   0   0  -4      0   0   0   0   0   0   0   0   0
  0  -1   0   0   0   0   0   0  -1      0   0   0   0   0   0   0   0   0
  0   0  -1   0   0   0   0   0  -1  …   0   0   0   0   0   0   0   0   0
  0   0   0  -1   0   0   0   0   0      0   0   0   0   0   0   0   0   0
  0   0   0   0  -1   0   0   0  -1      0   0   0   0   0   0   0   0   0
  

In [9]:
#this code generates a full basis for a Quantum Ising model of length N
"""
Binary `BitArray` representation of the given integer `num`, padded to length `N`.
"""
bit_rep(num::Integer, N::Integer) = BitArray(parse(Bool, i) for i in string(num, base=2, pad=N))

"""
    generate_basis(N::Integer) -> basis

Generates a basis (`Vector{BitArray}`) spanning the Hilbert space of `N` spins.
"""
function generate_basis(N::Integer)
    nstates = 2^N
    basis = Vector{BitArray{1}}(undef, nstates)
    for i in 0:nstates-1
        basis[i+1] = bit_rep(i, N)
    end
    return basis
end

generate_basis

In [10]:
generate_basis(2)

4-element Vector{BitVector}:
 [0, 0]
 [0, 1]
 [1, 0]
 [1, 1]

In [11]:
generate_basis(4)

16-element Vector{BitVector}:
 [0, 0, 0, 0]
 [0, 0, 0, 1]
 [0, 0, 1, 0]
 [0, 0, 1, 1]
 [0, 1, 0, 0]
 [0, 1, 0, 1]
 [0, 1, 1, 0]
 [0, 1, 1, 1]
 [1, 0, 0, 0]
 [1, 0, 0, 1]
 [1, 0, 1, 0]
 [1, 0, 1, 1]
 [1, 1, 0, 0]
 [1, 1, 0, 1]
 [1, 1, 1, 0]
 [1, 1, 1, 1]

In [12]:
#iterative construction of H

using LinearAlgebra

function TransverseFieldIsing_Explicit(; N::Integer, h::T=0) where T<:Real
    basis = generate_basis(N)
    H = zeros(T, 2^N, 2^N)
    bonds = zip(collect(1:N-1),collect(2:N))
    for (i, bstate) in enumerate(basis)
        #diagonal part
        diag_term = 0.0
        for (site_i, site_j) in bonds
            if bstate[site_i] == bstate[site_j]
                diag_term -= 1
            else
                diag_term += 1
            end
        end
        H[i,i] = diag_term
        
        #off diagonal part
        for site in 1:N
            new_bstate = copy(bstate)
            #flip the bit on the site (the action of σˣ on a bit)
            new_bstate[site] = !newb_state[site]
            #find corresponding single basis state with unity overlap (orthonormality)
            new_i = findfirst(isequal(new_bstate), basis)
            H[i, new_i] = -h
        end
    end
    return H
end

TransverseFieldIsing_Explicit (generic function with 1 method)

In [13]:
TransverseFieldIsing_Explicit(N=4, h=1)

LoadError: UndefVarError: `newb_state` not defined

In [14]:
basis = generate_basis(8)
H = TransverseFieldIsing(N=2, J=1, g=0)
vals, vecs = eigen(H)
groundstate = vecs[:,1]
abs2.(groundstate)

4-element Vector{Float64}:
 1.0
 0.0
 0.0
 0.0

In [15]:
basis[1]

8-element BitVector:
 0
 0
 0
 0
 0
 0
 0
 0

In [16]:
H = TransverseFieldIsing(N=8, J=1, g=100)
vals, vecs = eigen(H)
groundstate = vecs[:,1]
abs2.(groundstate)

256-element Vector{Float64}:
 0.004065668441957344
 0.0039849628943491745
 0.0039849628943491745
 0.003984762646227951
 0.003984962894349173
 0.0039060546850590533
 0.003984762646227947
 0.003984761646251895
 0.003984962894349173
 0.003905860366186066
 0.0039060546850590524
 0.00390585938110368
 0.003984762646227947
 ⋮
 0.003905859381103708
 0.0039060546850590958
 0.003905860366186104
 0.003984962894349253
 0.003984761646251924
 0.003984762646227994
 0.0039060546850590906
 0.003984962894349276
 0.003984762646227951
 0.003984962894349246
 0.003984962894349232
 0.004065668441957414

In [38]:
#Correlation function C(i,j)

function Sᶻᵢ(System_Size::Integer,spin_location::Integer)
    S = fill(id, System_Size)
    S[3] = σᶻ
    return S
end

S = Sᶻᵢ(4,2)


LoadError: InexactError: Int64(0.5)

Since we're looking for quantum magnets we will compute the overall magnetization, defined by 

$M = \frac{1}{N}\sum_{i}{\sigma^{z}_{i}}$

where $\sigma^{z}_{i}$ is the value of the spin on site $i$ when we measure

In [18]:
function magnetisation(state, basis)
    M = 0.0
    for (i, bstate) in enumerate(basis)
        bstate_M = 0
        for spin in bstate
            bstate_M += (state[i]^2 * (spin ? 1 : -1))/length(bstate)
        end
        @assert abs(bstate_M) <= 1
        M += abs(bstate_M)
    end
    return M
end

magnetisation (generic function with 1 method)

In [19]:
magnetisation(groundstate, basis)

0.2750078990794085

using Plots

ENV["GRDIR"]=""
Pkg.build("GR")

hs = 10 .^ range(-2., stop=2, length=10)
Ns = 2:10
p = plot()
for N in Ns
    M = zeros(length(hs))
    for (i,h) in enumerate(hs)
        basis = generate_basis(N)
        H = TransverseFieldIsing(N=N,J=1,g=h)
        vals, vecs = eigen(H)
        groundstate = vecs[:,1]
        M[i] = magnetisation(groundstate,basis)
    end
    plot!(p, hs, M, xscale=:log10, marker=:circle, label="N=$N", xlab="h", ylab="M(h)")
    println(M)
end